# HW3: Ẩn tin mật trên ảnh jpeg bằng phương pháp LSB

(Cập nhật lần cuối: 28/03/2023)

Họ tên: Nguyễn Quốc Huy

MSSV: 20127188

---

## Nắm cách làm bài và nộp bài

&#9889; Bạn lưu ý là mình sẽ dùng chương trình hỗ trợ chấm bài nên bạn cần phải tuân thủ chính xác qui định mà mình đặt ra, nếu không rõ thì hỏi, chứ không nên tự tiện làm theo ý của cá nhân.

**Cách làm bài**

Bạn sẽ làm trực tiếp trên file notebook này. Đầu tiên, bạn điền họ tên và MSSV vào phần đầu file ở bên trên. Trong file, bạn làm bài ở những chỗ có ghi là:
```python
# YOUR CODE HERE
raise NotImplementedError()
```
hoặc đối với những phần code không bắt buộc thì là:
```python
# YOUR CODE HERE (OPTION)
```
hoặc đối với markdown cell thì là:
```markdown
YOUR ANSWER HERE
```
Tất nhiên, khi làm thì bạn xóa dòng `raise NotImplementedError()` đi.
Đối những phần yêu cầu code thì thường ở ngay phía dưới sẽ có một (hoặc một số) cell chứa các bộ test để giúp bạn biết đã code đúng hay chưa; nếu chạy cell này không có lỗi gì thì có nghĩa là qua được các bộ test. Trong một số trường hợp, các bộ test có thể sẽ không đầy đủ; nghĩa là, nếu không qua được test thì là code sai, nhưng nếu qua được test thì chưa chắc đã đúng hoàn toàn.

Trong khi làm bài, bạn có thể cho in ra màn hình, tạo thêm các cell để test. Nhưng khi nộp bài thì bạn xóa các cell mà bạn tự tạo, xóa hoặc comment các câu lệnh in ra màn hình. Bạn lưu ý <font color=red>không được tự tiện xóa các cell hay sửa code của Thầy</font> (trừ những chỗ được phép sửa như đã nói ở trên).

Trong khi làm bài, thường xuyên `Ctrl + S` để lưu lại bài làm của bạn, tránh mất mát thông tin.

*Nên nhớ mục tiêu chính ở đây là <font color=green>học, học một cách chân thật</font>.  Bạn có thể thảo luận ý tưởng với bạn khác cũng như tham khảo các nguồn trên mạng, nhưng sau cùng <font color=green>code và bài làm phải là của bạn, dựa trên sự hiểu thật sự của bạn</font> (khi tham khảo các nguồn trên mạng thì bạn cần ghi rõ nguồn trong bài làm, và đương nhiên là bạn cũng không được phép đưa code và bài làm cho bạn khác xem). <font color=red>Nếu vi phạm những điều này thì có thể bạn sẽ bị 0 điểm cho toàn bộ môn học.</font>*

**Cách nộp bài**

Khi chấm bài, đầu tiên mình sẽ chọn `Kernel` - `Restart Kernel & Run All Cells`, để restart và chạy tất cả các cell trong notebook của bạn; do đó, trước khi nộp bài, bạn nên chạy thử `Kernel` - `Restart Kernel & Run All Cells` để đảm bảo mọi chuyện diễn ra đúng như mong đợi.

Sau đó, bạn tạo thư mục nộp bài theo cấu trúc sau:
- Thư mục `MSSV` (vd, nếu bạn có MSSV là 1234567 thì bạn đặt tên thư mục là `1234567`)
    - File `HW3.ipynb` (không cần nộp các file khác)

Cuối cùng, bạn nén thư mục `MSSV` này lại với định dạng nén là .zip (chứ không được là .rar hay các định dạng khác) và nộp ở link trên moodle. \
<font color=red>Bạn lưu ý tuân thủ chính xác qui định nộp bài này.</font>

## Kiểm tra môi trường code 

In [1]:
import sys
sys.executable

'c:\\ProgramData\\anaconda3\\envs\\dh_ss-env\\python.exe'

Bạn nên thấy kết quả in ra là đường dẫn đến file chạy python của môi trường "dh_ss-env" mà mình đã hướng dẫn bạn cài đặt ở HW0.

## Import các thư viện cần thiết

In [2]:
import constants
from huffman import Huffman 
import jpeg_decoder

import struct 
import numpy as np
from scipy import fftpack
from PIL import Image
import matplotlib.pyplot as plt

# YOUR CODE HERE (OPTION)

In [3]:
import importlib

def get_library_versions(libraries):
  """
  Retrieves the installed versions of specified libraries.

  Args:
      libraries (list): A list of library names (e.g., ["numpy", "scipy", "PIL"])

  Returns:
      dict: A dictionary mapping library names to their installed versions
              (e.g., {"numpy": "1.22.3", "scipy": "1.9.0", "PIL": "8.4.0"})
  """

  versions = {}
  for library in libraries:
    try:
      module = importlib.import_module(library)
      version = getattr(module, "__version__", None)
      versions[library] = version
    except ImportError:
      versions[library] = "Not installed"
  return versions

# Example usage
libraries = ["struct", "numpy", "scipy.fftpack", "PIL", "matplotlib"]
versions = get_library_versions(libraries)

print("Installed library versions:")
for library, version in versions.items():
  print(f"{library}: {version}")

Installed library versions:
struct: None
numpy: 1.24.2
scipy.fftpack: None
PIL: 9.4.0
matplotlib: 3.7.0


## Cài đặt các hàm hỗ trợ

In [4]:
# Ở HW1, ta dùng chuỗi bit (ví dụ: '101') 
# Ở HW này, ta sẽ dùng list các bit (ví dụ: [1, 0, 1]), vì như này sẽ dễ dàng hơn
# cho việc nhúng và rút trích trong HW này
def convert_s2bits(s):
    '''
    Chuyển chuỗi ký tự (ascii) thành list các bit.
    '''
    bits = []
    for c in s:
        c_bits = []
        for bit in list(bin(ord(c))[2:].zfill(8)):
            c_bits.append(int(bit))
        bits.extend(c_bits)
    return bits

def convert_bits2s(bits):
    '''
    Chuyển list các bit thành chuỗi ký tự (ascii).
    '''
    s = ''
    for i in range(0, len(bits), 8):
        c_bits = ''
        for bit in bits[i:i+8]:
            c_bits = c_bits + str(bit)
        c = chr(int(c_bits, 2))
        s = s + c
    return s

In [5]:
def do_dct(pixels):
    '''
    Hàm biến đổi từ ma trận điểm ảnh sang ma trận hệ số DCT (2 ma trận có cùng shape).
    '''
    return fftpack.dct(fftpack.dct(pixels, axis=0, norm='ortho'), axis=1, norm='ortho')

In [6]:
def get_header(img_height, img_width, quant_table):
    '''
    Hàm tính chuỗi byte ứng với header của ảnh JPEG.
    (Code được điều chỉnh từ nguồn: https://github.com/reinhrst/pygreypeg.)
    
    Bạn yên tâm, bạn không cần phải hiểu hàm này ;-).
    '''
    buf = bytearray()

    def writebyte(val):
        buf.extend(struct.pack(">B", val))

    def writeshort(val):
        buf.extend(struct.pack(">H", val))

    # SOI
    writeshort(0xFFD8)  # SOI marker

    # APP0
    writeshort(0xFFE0)  # APP0 marker
    writeshort(0x0010)  # segment length
    writebyte(0x4A)     # 'J'
    writebyte(0x46)     # 'F'
    writebyte(0x49)     # 'I'
    writebyte(0x46)     # 'F'
    writebyte(0x00)     # '\0'
    writeshort(0x0101)  # v1.1
    writebyte(0x00)     # no density unit
    writeshort(0x0001)  # X density = 1
    writeshort(0x0001)  # Y density = 1
    writebyte(0x00)     # thumbnail width = 0
    writebyte(0x00)     # thumbnail height = 0

    # DQT
    quant_table = quant_table.reshape(-1)
    writeshort(0xFFDB)  # DQT marker
    writeshort(0x0043)  # segment length
    writebyte(0x00)     # table 0, 8-bit precision (0)
    for index in constants.zz:
        writebyte(quant_table[index])

    # SOF0
    writeshort(0xFFC0)  # SOF0 marker
    writeshort(0x000B)  # segment length
    writebyte(0x08)     # 8-bit precision
    writeshort(img_height)
    writeshort(img_width)
    writebyte(0x01)     # 1 component only (grayscale)
    writebyte(0x01)     # component ID = 1
    writebyte(0x11)     # no subsampling
    writebyte(0x00)     # quantization table 0

    # DHT
    writeshort(0xFFC4)                     # DHT marker
    writeshort(19 + constants.dc_nb_vals)  # segment length
    writebyte(0x00)                        # table 0 (DC), type 0 (0 = Y, 1 = UV)
    for node in constants.dc_nodes[1:]:
        writebyte(node)
    for val in constants.dc_vals:
        writebyte(val)

    writeshort(0xFFC4)                     # DHT marker
    writeshort(19 + constants.ac_nb_vals)
    writebyte(0x10)                        # table 1 (AC), type 0 (0 = Y, 1 = UV)
    for node in constants.ac_nodes[1:]:
        writebyte(node)
    for val in constants.ac_vals:
        writebyte(val)

    # SOS
    writeshort(0xFFDA)  # SOS marker
    writeshort(8)       # segment length
    writebyte(0x01)     # nb. components
    writebyte(0x01)     # Y component ID
    writebyte(0x00)     # Y HT = 0
    # segment end
    writebyte(0x00)
    writebyte(0x3F)
    writebyte(0x00)

    return buf

In [7]:
# YOUR CODE HERE (OPTION)
def get_quant_table_index(quant_table):
    quant_index = list()
    for i, item in np.ndenumerate(quant_table):
        if item == 1:
            quant_index.append(i)
    return np.array(quant_index)

## Cài đặt hàm nhúng (6đ)

In [8]:
def embed(msg_file, cover_img_file, quant_table, stego_img_file):
    '''
    Nhúng tin mật vào ảnh jpeg (lossy) bằng phương pháp LSB với k = 1.
    Ta sẽ làm theo phương pháp của Chang, Chen, và Chung (bạn xem file slide lý 
    thuyết "06...", trang 37).
    Để đơn giản, ở đây ta sẽ giả định: ảnh là ảnh xám, có chiều dài và chiều 
    rộng chia hết cho 8.
    
    Các tham số:
        msg_file (str): Tên file chứa secret message.
        cover_img_file (str): Tên file chứa cover image.
        quant_table (mảng numpy 8x8): Bảng quantization (bảng các số chia ở bước quantization).
        stego_img_file (str): Tên file (*.jpg) chứa stego image (kết quả sau khi nhúng).
    Giá trị trả về:
        bool: True nếu nhúng thành không, False nếu không đủ chỗ để nhúng. 
    '''
    # I. Đọc cover img file
    cover_pixels = np.array(Image.open(cover_img_file))
    # print(cover_pixels.dtype) # Lưu ý: cover_pixels có dtype là uint8!
    height, width = cover_pixels.shape
    if height % 8 != 0 or width % 8 != 0:
        print('For simplicity, we just work with image having height and width multiple of 8')
        return False
    
    # II. Đọc msg file, chuyển msg thành msg bits, kiểm xem có đủ chỗ nhúng không, 
    #     thêm 100... vào msg bits (có thể bạn sẽ muốn chuyển kết quả sau cùng từ 
    #     list thành mảng numpy để lúc sau có thể dễ xử lý hơn)
    # YOUR CODE HERE
    with open(msg_file, 'r') as f:
        msg = f.read()
        
    # Chuyển msg thành msg bits
    bit = convert_s2bits(msg)
    msg_bits = ''.join(str(x) for x in bit)
    
    quant_table_index = get_quant_table_index(quant_table)
    
    # Kiểm tra sức chứa
    msg_size = len(msg_bits)
    capacity = (cover_pixels.size / 64) * len(quant_table_index)
    if msg_size + 1 > int(capacity):
        return False
    
    # Thêm 100... vào msg bits
    msg_bits += '1' + '0' * (int(capacity) - msg_size - 1)
   
    # III. Nén jpeg, trong quá trình nén thực hiện nhúng msg bits
    jpeg_bytes = bytearray()
    jpeg_bytes.extend(get_header(height, width, quant_table))
    huf = Huffman()
    
    # Lần lượt duyệt các khối ảnh 8x8 (theo thứ tự từ trái qua phải, từ trên xuống dưới)
    # Với mỗi khối:
    # (1) Trừ 128 (lưu ý kiểu dữ liệu!), rồi tính các hệ số DCT (dùng hàm `do_dct`)
    # (2) Tính các hệ số quantized DCT
    # (3) Nhúng msg bits vào các hệ số quantized DCT
    # (4) Nén các hệ số quantized DCT bằng thuật toán nén Huffman
    #     Để nén, bạn dùng câu lệnh `huf.encode_block(zz_quant_dct_coefs, length)`
    #     (kết quả nén sẽ được lưu lại bên trong đối tượng `huf`)
    #     Trong đó: 
    #     - `zz_quant_dct_coefs` là mảng 1 chiều chứa các hệ số quantized DCT 
    #       (có được bằng cách duyệt mảng 2 chiều theo thứ tự dích dắc:
    #       đầu tiên, nối các dòng của mảng 2 chiều để có được mảng một chiều, 
    #       rồi duyệt mảng một chiều này theo mảng chỉ số `constants.zz` đã được định nghĩa sẵn cho bạn)
    #     - `length` là số lượng phần tử của mảng `zz_quant_dct_coefs` tính
    #       từ phần tử đầu cho đến phần tử khác 0 cuối cùng 
    #       (lưu ý: có thể xảy ra trường hợp tất cả phần tử đều bằng 0)
    # YOUR CODE HERE
    # Tạo mảng 8x8 
    blocks_list = list()
    row = 0
    while row < 512:
        col = 0
        while col < 512:
            position = (row, col)
            block = cover_pixels[position[0]:position[0]+8, position[1]:position[1]+8]
            blocks_list.append(block)
            col = col + 8
        row = row + 8
    np_blocks = np.array(blocks_list).astype(int)
    
    # Duyệt và xử lý qua từng mảng 8x8
    msg_bit_index  = 0
    num_lsbs = 1
    for block in np_blocks: 
        subtracted_block = block - 128
        dct_coefficients = do_dct(subtracted_block)
        quantized_dct = dct_coefficients / quant_table
        rounded_quantized_dct = np.rint(quantized_dct)
        
        stego_block = np.array(rounded_quantized_dct).astype(int)
        # Nhúng bits vào các hệ số quantized DCT
        for index, item in np.ndenumerate(rounded_quantized_dct):
            item_parse = int(item)
            if (index == quant_table_index).all(1).any():
                bit_to_embed = int(bin(int(msg_bits[msg_bit_index:msg_bit_index + num_lsbs], 2))[2:], 2)
                embedded_value = int((item_parse>>num_lsbs<<num_lsbs) + bit_to_embed)
                msg_bit_index = msg_bit_index + num_lsbs
                stego_block[index] = embedded_value
    
        # Tạo mảng 1 chiều chứa các hệ số quantized DC
        stego_block_reshape = stego_block.reshape(-1)
        zz_quant_dct_coefs = np.empty(0, dtype=int)
        for index in constants.zz:
            zz_quant_dct_coefs = np.append(zz_quant_dct_coefs, stego_block_reshape[index])
    
        # Tính length (số lượng phần tử của mảng `quant_dct_coefs` tính từ phần tử đầu cho đến phần tử khác 0 cuối cùng)
        indices = np.argwhere(zz_quant_dct_coefs != 0)
        last_index = indices[-1]
        length = last_index[0] + 1
        huf.encode_block(zz_quant_dct_coefs, length)
    
    jpeg_bytes.extend(huf.end_and_get_buffer())
    jpeg_bytes.extend(struct.pack(">H", 0xFFD9))  # EOI marker
    
    # IV. Ghi kết quả nén jpeg xuống file
    with open(stego_img_file, 'wb') as f:
        f.write(jpeg_bytes)
    
    return True

In [9]:
# TEST
quant_table = np.array([
    16, 11, 10, 16,  1,  1,  1,  1,
    12, 12, 14,  1,  1,  1,  1, 55,
    14, 13,  1,  1,  1,  1, 69, 56,
    14,  1,  1,  1,  1, 87, 80, 62,
     1,  1,  1,  1, 68, 109, 103, 77,
     1,  1,  1, 64, 81, 104, 113, 92,
     1,  1, 78, 87, 103, 121, 120, 101,
     1, 92, 95, 98, 112, 100, 103, 99
]).reshape(8, 8)
result = embed('msg2.txt', 'cover.bmp', quant_table, 'stego.jpg')
assert result == False

In [ ]:
# TEST
result = embed('msg.txt', 'cover.bmp', quant_table, 'stego.jpg')
assert result == True
assert np.all(np.array(Image.open('stego.jpg')) == np.array(Image.open('correct_stego.jpg')))

## Cài đặt hàm rút trích (4đ)

In [11]:
def extract(stego_img_file, extr_msg_file):
    '''
    Hàm rút trích tin mật đã được nhúng vào ảnh jpeg.
    
    Các tham số:
        stego_img_file (str): Tên file chứa stego image.
        extr_msg_file (str): Tên file chứa secret message được rút trích.
    '''
    # Trong quá trình giải nén stego img file, lấy các hệ số quantized dct và bảng quatization
    all_quant_dct_coefs, quant_table = jpeg_decoder.get_quant_dct_coefs_and_quant_table(stego_img_file)
    # all_quant_dct_coefs là mảng một chiều chứa các hệ số quantized dct của toàn ảnh
    # all_quant_dct_coefs có được bằng cách nối các mảng một chiều (không dích dắc) 
    # chứa các hệ số quantized dct của các khối ảnh 8x8
    # print(quant_dct_coefs.shape)
    
    # Phần còn lại là của bạn
    # YOUR CODE HERE
    # Chia bảng hệ số thành các block 8x8
    table_factor_reshape = all_quant_dct_coefs.reshape(-1,8,8)
    
    quant_table_index = get_quant_table_index(quant_table)

    num_lsbs = 1
    extract_bit = '' 
    # Duyệt từng block 
    for block in table_factor_reshape: 
        block_bits = '' 
        # Duyệt từng phần tử trong block
        for index, item in np.ndenumerate(block): 
            # Nếu phần tử có index nằm trong danh sách CCC thì xét tiếp
            if (index == quant_table_index).all(1).any():
                block_bits += str(np.binary_repr(abs(item) & (2**num_lsbs-1), num_lsbs))[-1] 
        extract_bit += block_bits 

    # Tìm đuôi 100.. để rút trích msg bits
    extract_bit = extract_bit[:extract_bit.rfind('1')]
    print(extract_bit)
    # Chuyển dãy bit thành chuỗi
    extract_msg = convert_bits2s(extract_bit)
    
    # Ghi ra file
    with open(extr_msg_file, 'w') as f:
        f.write(extract_msg)

In [12]:
# TEST
extract('stego.jpg', 'extr_msg.txt')
with open('extr_msg.txt', 'r') as f:
    extr_msg = f.read()
with open('msg.txt', 'r') as f:
    correct_extr_msg = f.read()
assert extr_msg == correct_extr_msg

0100111101010110010001010101001000100000010110010100111101010101000010100101101101010111011000010111010001100011011010000010000001101001011101000010000001101000011001010111001001100101001110100010000000001010011010000111010001110100011100000111001100111010001011110010111101110111011101110111011100101110011110010110111101110101011101000111010101100010011001010010111001100011011011110110110100101111011101110110000101110100011000110110100000111111011101100011110101111001001100100011010001000100010011000100111101100100010001100111000001110100001100000010000000101000011000100111100100100000010000110110000101110011011100110110000101100100011001010110010100100000010100000110111101110000011001010010100100001010011010000111010001110100011100000111001100111010001011110010111101110111011101110111011100101110011110010110111101110101011101000111010101100010011001010010111001100011011011110110110100101111011101110110000101110100011000110110100000111111011101100011110101100011010001110100000101000010